In [2]:
import pandas as pd
import numpy as np

train_df = pd.read_csv('data/train.csv').drop(columns=['id', 'lang_abv','language'])
test_df = pd.read_csv('data/test.csv').drop(columns=['id', 'lang_abv', 'language'])                 


In [3]:
from datasets import Dataset

#Convert DataFrame to Dataset
train_dataset = Dataset.from_pandas(train_df, split="train")
test_dataset = Dataset.from_pandas(test_df, split="test")

c:\Users\vlns\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-cased")
# Tokenize both 'premise' and 'hypothesis' columns
def tokenize_function(examples):
    return tokenizer(
        text=[pre + tokenizer.sep_token + hyp for pre, hyp in zip(examples["premise"], examples["hypothesis"])],
        padding="max_length",
        truncation=True
    )

In [20]:
tokenized_train = train_dataset.map(tokenize_function, batched=True)
tokenized_test = test_dataset.map(tokenize_function, batched=True)

Map: 100%|██████████| 5195/5195 [00:02<00:00, 2597.45 examples/s]


In [9]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("bert-base-multilingual-cased", num_labels=3)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
from transformers import TrainingArguments

training_args = TrainingArguments(output_dir="test_trainer")

In [11]:
import numpy as np
import evaluate

metric = evaluate.load("accuracy")

In [12]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [13]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch")

In [21]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    compute_metrics=compute_metrics,
)

c:\Users\vlns\Python\Python311\Lib\site-packages\accelerate\accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(
  0%|          | 0/3 [04:13<?, ?it/s]


In [22]:
trainer.train()

  0%|          | 9/4545 [03:13<28:00:40, 22.23s/it]